In [18]:
import pandas as pd
import numpy as np
import statsmodels
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LassoCV

In [19]:
df = pd.read_csv('C:/Users/think/Downloads/tot_daily_df_interpolate.csv')
df.set_index('DATE',inplace=True)
df.dropna(how='any',inplace=True)

In [20]:
# Only the diff of 'T10Y2Y', 'DGS10','RU3000TRReturn', 'UNRATE' are stationary, but we don't need stationary for OLS and feature selection
'''
for i in ['T10Y2Y','DGS10','RU3000TRReturn','UNRATE']:
    df['diff'+i]=pd.DataFrame(df[i].diff(periods=1))
print (df.columns)
df.dropna(how='any',inplace=True)
'''

"\nfor i in ['T10Y2Y','DGS10','RU3000TRReturn','UNRATE']:\n    df['diff'+i]=pd.DataFrame(df[i].diff(periods=1))\nprint (df.columns)\ndf.dropna(how='any',inplace=True)\n"

In [21]:
# Test for stationary with ADF test
adfResult=[]
for c in df.columns:
    adfTest = adfuller(df[c], autolag='AIC')
    adfResult.append(pd.Series(adfTest[0:4]))
adfResults=pd.concat(adfResult,axis=1,sort=False)
adfResults.columns=df.columns
adfResults.index= ['ADF Test Statistic', 'P-Value', '# Lags Used', '# Observations Used']
print (adfResults)

                          T10Y2Y        DGS10       AAA10Y       VIXCLS  \
ADF Test Statistic     -1.638230    -1.755908    -3.043184    -4.647704   
P-Value                 0.463186     0.402561     0.031054     0.000105   
# Lags Used             5.000000     7.000000    21.000000    35.000000   
# Observations Used  6733.000000  6731.000000  6717.000000  6703.000000   

                       GoldReturn  DCOILWTICOReturn  NASDAQ100Return  \
ADF Test Statistic  -1.643694e+01        -19.115509    -1.451530e+01   
P-Value              2.448325e-29          0.000000     5.637766e-27   
# Lags Used          2.900000e+01         16.000000     2.600000e+01   
# Observations Used  6.709000e+03       6722.000000     6.712000e+03   

                     RU2000PRReturn  RU3000TRReturn       UNRATE  \
ADF Test Statistic    -1.449996e+01       -1.966044    -1.088968   
P-Value                5.954743e-27        0.301657     0.719451   
# Lags Used            3.200000e+01       25.000000    35.0

In [22]:
# Detect Collinearity, delete all fratures with vif>5 one by one
vif = pd.DataFrame()
dfnew=df.drop('AAA10Y',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(dfnew.values, i) for i in range(dfnew.shape[1])]
vif["features"] = dfnew.columns
print (vif)

vif = pd.DataFrame()
dfnew=dfnew.drop('UNRATE',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(dfnew.values, i) for i in range(dfnew.shape[1])]
vif["features"] = dfnew.columns
print (vif)

vif = pd.DataFrame()
dfnew=dfnew.drop('VIXCLS',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(dfnew.values, i) for i in range(dfnew.shape[1])]
vif["features"] = dfnew.columns
print (vif)

vif = pd.DataFrame()
dfnew=dfnew.drop('RU3000TRReturn',axis=1)
vif["VIF Factor"] = [variance_inflation_factor(dfnew.values, i) for i in range(dfnew.shape[1])]
vif["features"] = dfnew.columns
print (vif)

    VIF Factor          features
0     7.774515            T10Y2Y
1     6.643860             DGS10
2    11.826009            VIXCLS
3     1.004308        GoldReturn
4     1.044560  DCOILWTICOReturn
5     2.402582   NASDAQ100Return
6     2.491236    RU2000PRReturn
7    11.991107    RU3000TRReturn
8    26.306097            UNRATE
9     1.875412      BUSINVReturn
10    2.025225    CPIAUCSLReturn
   VIF Factor          features
0    2.617686            T10Y2Y
1    5.517197             DGS10
2   11.113350            VIXCLS
3    1.004263        GoldReturn
4    1.044538  DCOILWTICOReturn
5    2.402574   NASDAQ100Return
6    2.490578    RU2000PRReturn
7   10.873024    RU3000TRReturn
8    1.787624      BUSINVReturn
9    2.006611    CPIAUCSLReturn
   VIF Factor          features
0    2.355783            T10Y2Y
1    4.693292             DGS10
2    1.003960        GoldReturn
3    1.044457  DCOILWTICOReturn
4    2.402503   NASDAQ100Return
5    2.455965    RU2000PRReturn
6    5.368091    RU3000TRRet

In [23]:
# Feature Selection with LASSO, we want the features with coef not zero
X1, y=df.drop('AAA10Y',axis=1),df['AAA10Y']
X=dfnew
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X, y)
coef = pd.Series(model_lasso.coef_, index = X.columns)
print (coef)
# final answer: T10Y2Y,DGS10,BUSINVReturn

T10Y2Y              0.190924
DGS10              -0.119744
GoldReturn         -0.000000
DCOILWTICOReturn   -0.000000
NASDAQ100Return    -0.000000
RU2000PRReturn     -0.000000
BUSINVReturn       -9.048723
CPIAUCSLReturn     -0.000000
dtype: float64


C:\Users\think\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
